In [ ]:
import pickle
# from collections import defaultdict
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
file_path = 'sim_results.pkl'

with open(file_path, 'rb') as f:
    results = pickle.load(f)

In [ ]:
attributes = ['STR','DEX','CON','INT','WIS','CHA']

attribute_map = {
    0: 'STR',
    1: 'DEX',
    2: 'CON',
    3: 'INT',
    4: 'WIS',
    5: 'CHA'
}

attribute_scores = {}
# Iterate through the 50 iterations (results) for the 10,000 characters
for i in range(50):
    # get all attribute arrays of the simulation
    stat_arrays = results[i][8]
    #initiate a list to hold all the dictionary stat arrays
    attribute_scores[i] = []
    for char_id, char_stats in stat_arrays.items():
        attribute_scores[i].append(char_stats)

In [ ]:
correlations = {}
# Iterate over 50 sets of results
for i in tqdm(range(50)):
    # Get the ith set of attribute dictionaries
    data = attribute_scores[i]  
    # Initiate a dictionary to hold six empty lists
    atts = {j: [] for j in range(6)} 
    
    # extract all attributes to individual lists
    for d in data:
        for j in range(6):
            atts[j].append(d[j])  
    
    # initiate 
    if i not in correlations:
        correlations[i] = {}
    
    for j in range(6):
        for k in range(6):
            correlations[i][j, k] = np.corrcoef(atts[j], atts[k])[0, 1]

In [ ]:
# Function to calculate Fisher Z-transformation
def fisher_z(correlation):
    return 0.5 * np.log((1 + correlation) / (1 - correlation))

# Function to inverse Fisher Z-transformation
def inverse_fisher_z(z):
    return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)

# Function to calculate 95% confidence intervals for each correlation coefficient
def calculate_ci(correlation_values, alpha=0.05):
    n = len(correlation_values)  # Number of samples (50 in your case)
    
    # Apply Fisher Z-transformation to the correlation values
    z_values = np.array([fisher_z(r) for r in correlation_values])
    
    # Calculate the mean and standard error of the Z values
    z_mean = np.mean(z_values)
    z_se = np.std(z_values, ddof=1) / np.sqrt(n)
    
    # Calculate the confidence interval in the Z space
    z_ci_lower = z_mean - 1.96 * z_se
    z_ci_upper = z_mean + 1.96 * z_se
    
    # Inverse Fisher Z-transformation to get the CI in the correlation space
    r_ci_lower = inverse_fisher_z(z_ci_lower)
    r_ci_upper = inverse_fisher_z(z_ci_upper)
    
    return r_ci_lower, r_ci_upper

# Initialize the dictionary with your 50 correlation matrices (assuming 'correlations' is the dictionary with your 50 matrices)
confidence_intervals = np.zeros((6, 6, 2))  # 6x6 matrix with 2 columns (lower and upper bounds)

# Iterate over each pair of attributes (j, k)
for j in range(6):  # Loop through the rows (attributes)
    for k in range(6):  # Loop through the columns (attributes)
        # Collect the correlation values for this pair across all 50 datasets
        correlation_values = [correlations[i].get((j, k), np.nan) for i in range(50)]
        
        # Remove NaN values (in case there were any missing correlations)
        correlation_values = [r for r in correlation_values if not np.isnan(r)]
        
        # Calculate the CI for this pair of attributes
        ci_lower, ci_upper = calculate_ci(correlation_values)
        
        # Store the CI in the matrix (lower and upper bounds)
        confidence_intervals[j, k, 0] = ci_lower  # Lower bound
        confidence_intervals[j, k, 1] = ci_upper

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the attribute labels
attribute_labels = ['STR', 'DEX', 'CON', 'INT', 'WIS', 'CHA']

# Assuming confidence_intervals is your 6x6 array containing the CI lower and upper bounds
# You will take the average of the lower and upper bounds to represent the final correlation value

# Calculate the average correlation value for each pair (j, k)
average_correlation_matrix = np.zeros((6, 6))

for j in range(6):
    for k in range(6):
        lower_bound = confidence_intervals[j, k, 0]
        upper_bound = confidence_intervals[j, k, 1]
        
        average_correlation_matrix[j, k] = (lower_bound + upper_bound) / 2
np.fill_diagonal(average_correlation_matrix, 1)


fig, ax = plt.subplots(figsize = (8,6))
# plt.figure(figsize=(8, 6))
cax = plt.imshow(
    average_correlation_matrix,
    cmap='coolwarm',
    interpolation='nearest',
    vmin = -1, vmax = 1
)
cbar = fig.colorbar(cax)

cbar.ax.tick_params(labelsize = 14) 
cbar.set_label('Correlation Coefficient',fontsize = 19)
# Add colorbar to the side
# plt.colorbar(cax, label='Correlation Coefficient')


# Set ticks and labels
plt.xticks(np.arange(6), attribute_labels, fontsize = 14)
plt.yticks(np.arange(6), attribute_labels, fontsize = 14)

# Add labels and title
plt.title('Attribute Correlation Matrix', fontsize = 22)
plt.xlabel('Attributes', fontsize = 19)
plt.ylabel('Attributes', fontsize = 19)

for i in range(6):
    for j in range(6):
        plt.text(j, i,
                 f'{average_correlation_matrix[i, j]:.2f}',
                 ha = 'center', va = 'center', 
                 color = 'black' if average_correlation_matrix[i, j] < 0.5 else 'white',
                 fontsize = '14'
                )

# Show the plot
plt.tight_layout()
plt.savefig('figs/attribute_correlation.png')
plt.show()


In [ ]:
average_correlation_matrix